<a href="https://colab.research.google.com/github/GabrielBG0/Fakenews-Recognition/blob/main/Bart_%26_FNN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Deleting default sample data folder
!rm -rf /content/sample_data
!unzip FakeNewsNet.zip

Archive:  FakeNewsNet.zip
  inflating: FakeNewsNet/dict_documents.pkl  
  inflating: FakeNewsNet/indexes.npy  
  inflating: FakeNewsNet/labels.npy  


In [2]:
# Install aditional librarys
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.1MB 9.6MB/s 
     |████████████████████████████████| 3.3MB 56.1MB/s 
     |████████████████████████████████| 901kB 31.6MB/s 
     |████████████████████████████████| 204kB 8.5MB/s 
     |████████████████████████████████| 112kB 8.9MB/s 
     |████████████████████████████████| 245kB 13.3MB/s 


In [3]:
import pickle
import numpy as np


documents = pickle.load(open("FakeNewsNet/dict_documents.pkl", "rb"))
indexes = np.load('FakeNewsNet/indexes.npy')
lbls = np.load('FakeNewsNet/labels.npy')

texts = []
labels = []
for i in range(len(indexes)):
    index = indexes[i]
    texts.append(documents[index])
    if lbls[i] == np.str_("-1"):
      labels.append(0)
    else :
      labels.append(1)

print("lable 1: " + str(labels[0]))
print("text 1: " + texts[0])
print(type(labels[0]))

lable 1: 0
text 1: When I revealed in this column last year that Dame Helen Mirren was the new face of L’Oreal, I had hundreds of letters from readers, ecstatic that beauty giants are celebrating older women.  As these exclusive pictures show, from next week (February 5) Helen will be appearing in her first television advertising campaign for the company.  The 69-year-old actress will be seen championing the role of the older woman — and the candid, unretouched shots from the advert show that wrinkles, ageing hands and natural make-up can be beautiful.  Scroll down for video  Dame Helen Mirren photographed for the L'Oreal Paris age perfect campaign. In this shot Helen wears the Age Perfect day cream, the candid, unretouched shots are a refreshing change and champion the beauty of the older woman  I suspect this approach will sell more products than the glossier, airbrushed campaigns that many women feel don’t relate to them and their real lives.  During the filming of the advert, Dame 

In [4]:
for text in texts:
  if len(text.split(' ')) > 512:
    print("there are too long texts! truncate then")
    break

there are too long texts! truncate then


In [5]:
import torch
from torch.utils.data import (TensorDataset, DataLoader,
                              RandomSampler, SequentialSampler)

from transformers import BertTokenizer, BertConfig
from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

from distutils.version import LooseVersion as LV

from sklearn.model_selection import train_test_split

import io, sys, os, datetime

import numpy as np

In [6]:
if 'DATADIR' in os.environ:
    DATADIR = os.environ['DATADIR']
else:
    DATADIR = "/content/"

In [7]:
# Split the data into a training set and a test set using
# scikit-learn's train_test_split().

TEST_SET = 4000

(sentences_train, sentences_test,
 labels_train, labels_test) = train_test_split(texts, labels,
                                               test_size=TEST_SET,
                                               shuffle=True, random_state=42)

print('Length of training texts:', len(sentences_train))
print('Length of training labels:', len(labels_train))
print('Length of test texts:', len(sentences_test))
print('Length of test labels:', len(labels_test))

Length of training texts: 3003
Length of training labels: 3003
Length of test texts: 4000
Length of test labels: 4000


In [8]:
# The token [CLS] is a special token required by BERT at the beginning
# of the sentence.

sentences_train = ["[CLS] " + s for s in sentences_train]
sentences_test = ["[CLS] " + s for s in sentences_test]

print ("The first training sentence:")
print(sentences_train[0], 'LABEL:', labels_train[0])

The first training sentence:
[CLS] The Real Housewives of Atlanta family just welcomed a new member. RHOA rookie Eva Marcille gave birth to a baby boy, Michael Todd Sterling, Jr., on Friday, April 13, she shared on Instagram.  Eva's son was born at 5:59 p.m. on Friday at 7 pounds, 3 ounces and 19-and-a-half inches long, according to a post on Instagram that showed a photo of little Michael's footprints. Eva captioned the photo, "All praises be to God!"  The baby boy is Eva's first child with her fiancé, Michael Todd Sterling, an attorney and former candidate for mayor of Atlanta, and their child is named after Sterling. She also has a 4-year-old daughter, Marley Rae, from a previous relationship with musician Kevin McCall.  🤱🏽 All praises be to God! A post shared by Eva Marcille (@evamarcille) on Apr 13, 2018 at 5:57pm PDT  It appears that Eva has been in the hospital since Thursday, April 12, according to photos she posted on Instagram.  Still at it😳 Come on Mikey🤰🏽 A post shared by E

In [9]:
# Next we specify the pre-trained BERT model we are going to use. The
# model `"bert-base-uncased"` is the lowercased "base" model
# (12-layer, 768-hidden, 12-heads, 110M parameters).
#
# We load the used vocabulary from the BERT model, and use the BERT
# tokenizer to convert the sentences into tokens that match the data
# the BERT model was trained on.

print('Initializing BertTokenizer')

BERTMODEL='bert-base-uncased'
CACHE_DIR=os.path.join(DATADIR, 'transformers-cache')

tokenizer = BertTokenizer.from_pretrained(BERTMODEL, cache_dir=CACHE_DIR,
                                          do_lower_case=True)

tokenized_train = [tokenizer.tokenize(s) for s in sentences_train]
tokenized_test  = [tokenizer.tokenize(s) for s in sentences_test]

print ("The full tokenized first training sentence:")
print (tokenized_train[0])

Initializing BertTokenizer



The full tokenized first training sentence:
['[CLS]', 'the', 'real', 'house', '##wives', 'of', 'atlanta', 'family', 'just', 'welcomed', 'a', 'new', 'member', '.', 'r', '##ho', '##a', 'rookie', 'eva', 'marc', '##ille', 'gave', 'birth', 'to', 'a', 'baby', 'boy', ',', 'michael', 'todd', 'sterling', ',', 'jr', '.', ',', 'on', 'friday', ',', 'april', '13', ',', 'she', 'shared', 'on', 'ins', '##tagram', '.', 'eva', "'", 's', 'son', 'was', 'born', 'at', '5', ':', '59', 'p', '.', 'm', '.', 'on', 'friday', 'at', '7', 'pounds', ',', '3', 'ounce', '##s', 'and', '19', '-', 'and', '-', 'a', '-', 'half', 'inches', 'long', ',', 'according', 'to', 'a', 'post', 'on', 'ins', '##tagram', 'that', 'showed', 'a', 'photo', 'of', 'little', 'michael', "'", 's', 'footprints', '.', 'eva', 'capt', '##ioned', 'the', 'photo', ',', '"', 'all', 'praises', 'be', 'to', 'god', '!', '"', 'the', 'baby', 'boy', 'is', 'eva', "'", 's', 'first', 'child', 'with', 'her', 'fiance', ',', 'michael', 'todd', 'sterling', ',', 'an',

In [10]:
# Now we set the maximum sequence lengths for our training and test
# sentences as `MAX_LEN_TRAIN` and `MAX_LEN_TEST`. The maximum length
# supported by the used BERT model is 512.
#
# The token `[SEP]` is another special token required by BERT at the
# end of the sentence.

MAX_LEN_TRAIN, MAX_LEN_TEST = 128, 512

tokenized_train = [t[:(MAX_LEN_TRAIN-1)]+['SEP'] for t in tokenized_train]
tokenized_test  = [t[:(MAX_LEN_TEST-1)]+['SEP'] for t in tokenized_test]

print ("The truncated tokenized first training sentence:")
print (tokenized_train[0])

The truncated tokenized first training sentence:
['[CLS]', 'the', 'real', 'house', '##wives', 'of', 'atlanta', 'family', 'just', 'welcomed', 'a', 'new', 'member', '.', 'r', '##ho', '##a', 'rookie', 'eva', 'marc', '##ille', 'gave', 'birth', 'to', 'a', 'baby', 'boy', ',', 'michael', 'todd', 'sterling', ',', 'jr', '.', ',', 'on', 'friday', ',', 'april', '13', ',', 'she', 'shared', 'on', 'ins', '##tagram', '.', 'eva', "'", 's', 'son', 'was', 'born', 'at', '5', ':', '59', 'p', '.', 'm', '.', 'on', 'friday', 'at', '7', 'pounds', ',', '3', 'ounce', '##s', 'and', '19', '-', 'and', '-', 'a', '-', 'half', 'inches', 'long', ',', 'according', 'to', 'a', 'post', 'on', 'ins', '##tagram', 'that', 'showed', 'a', 'photo', 'of', 'little', 'michael', "'", 's', 'footprints', '.', 'eva', 'capt', '##ioned', 'the', 'photo', ',', '"', 'all', 'praises', 'be', 'to', 'god', '!', '"', 'the', 'baby', 'boy', 'is', 'eva', "'", 's', 'first', 'child', 'with', 'her', 'fiance', ',', 'michael', 'SEP']


In [11]:
# Next we use the BERT tokenizer to convert each token into an integer
# index in the BERT vocabulary. We also pad any shorter sequences to
# `MAX_LEN_TRAIN` or `MAX_LEN_TEST` indices with trailing zeros.

ids_train = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_train]
ids_train = np.array([np.pad(i, (0, MAX_LEN_TRAIN-len(i)),
                             mode='constant') for i in ids_train])

ids_test = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_test]
ids_test = np.array([np.pad(i, (0, MAX_LEN_TEST-len(i)),
                            mode='constant') for i in ids_test])

print ("The indices of the first training sentence:")
print (ids_train[0])

The indices of the first training sentence:
[  101  1996  2613  2160 23744  1997  5865  2155  2074 10979  1037  2047
  2266  1012  1054  6806  2050  8305  9345  7871 10484  2435  4182  2000
  1037  3336  2879  1010  2745  6927 10933  1010  3781  1012  1010  2006
  5958  1010  2258  2410  1010  2016  4207  2006 16021 23091  1012  9345
  1005  1055  2365  2001  2141  2012  1019  1024  5354  1052  1012  1049
  1012  2006  5958  2012  1021  7038  1010  1017 19471  2015  1998  2539
  1011  1998  1011  1037  1011  2431  5282  2146  1010  2429  2000  1037
  2695  2006 16021 23091  2008  3662  1037  6302  1997  2210  2745  1005
  1055 24629  1012  9345 14408 19798  1996  6302  1010  1000  2035 27128
  2022  2000  2643   999  1000  1996  3336  2879  2003  9345  1005  1055
  2034  2775  2007  2014 19154  1010  2745   100]


In [12]:
# BERT also requires *attention masks*, with 1 for each real token in
# the sequences and 0 for the padding:

amasks_train, amasks_test = [], []

for seq in ids_train:
  seq_mask = [float(i>0) for i in seq]
  amasks_train.append(seq_mask)

for seq in ids_test:
  seq_mask = [float(i>0) for i in seq]
  amasks_test.append(seq_mask)

In [13]:
# We use again scikit-learn's train_test_split to use 10% of our
# training data as a validation set, and then convert all data into
# torch.tensors.

(train_inputs, validation_inputs,
 train_labels, validation_labels) = train_test_split(ids_train, labels_train,
                                                     random_state=42,
                                                     test_size=0.1)
(train_masks, validation_masks,
 _, _) = train_test_split(amasks_train, ids_train,
                          random_state=42, test_size=0.1)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks  = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks  = torch.tensor(validation_masks)
test_inputs = torch.tensor(ids_test)
test_labels = torch.tensor(labels_test)
test_masks  = torch.tensor(amasks_test)

In [14]:
# Next we create PyTorch DataLoaders for all data sets.
#
# For fine-tuning BERT on a specific task, the authors recommend a
# batch size of 16 or 32.

BATCH_SIZE = 32

print('Train: ', end="")
train_data = TensorDataset(train_inputs, train_masks,
                           train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler,
                              batch_size=BATCH_SIZE)
print(len(train_data), 'messages')

print('Validation: ', end="")
validation_data = TensorDataset(validation_inputs, validation_masks,
                                validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data,
                                   sampler=validation_sampler,
                                   batch_size=BATCH_SIZE)
print(len(validation_data), 'messages')

print('Test: ', end="")
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler,
                             batch_size=BATCH_SIZE)
print(len(test_data), 'messages')

Train: 2702 messages
Validation: 301 messages
Test: 4000 messages


In [15]:
# ## BERT model initialization
#
# We now load a pretrained BERT model with a single linear
# classification layer added on top.

print('Initializing BertForSequenceClassification')

model = BertForSequenceClassification.from_pretrained(BERTMODEL,
                                                      cache_dir=CACHE_DIR,
                                                      num_labels=20)
model.cuda()

Initializing BertForSequenceClassification


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
# We set the remaining hyperparameters needed for fine-tuning the
# pretrained model:
#   * EPOCHS: the number of training epochs in fine-tuning
#     (recommended values between 2 and 4)
#   * WEIGHT_DECAY: weight decay for the Adam optimizer
#   * LR: learning rate for the Adam optimizer (2e-5 to 5e-5 recommended)
#   * WARMUP_STEPS: number of warmup steps to (linearly) reach the set
#     learning rate
#
# We also need to grab the training parameters from the pretrained model.

EPOCHS = 16
WEIGHT_DECAY = 0.01
LR = 2e-5
WARMUP_STEPS =int(0.2*len(train_dataloader))

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)],
     'weight_decay': WEIGHT_DECAY},
    {'params': [p for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)],
     'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=LR, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=WARMUP_STEPS,
                                            num_training_steps=len(train_dataloader)*EPOCHS)

In [18]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [19]:
# ## Learning
#
# Let's now define functions to train() and evaluate() the model:

def train(epoch, loss_vector=None, log_interval=200):
  # Set model to training mode
  model.train()

  # Loop over each batch from the training set
  for step, batch in enumerate(train_dataloader):

    # Copy data to GPU if needed
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Zero gradient buffers
    optimizer.zero_grad()

    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None,
                    attention_mask=b_input_mask, labels=b_labels)

    loss = outputs[0]
    if loss_vector is not None:
        loss_vector.append(loss.item())

    # Backward pass
    loss.backward()

    # Update weights
    optimizer.step()
    scheduler.step()

    if step % log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, step * len(b_input_ids),
                len(train_dataloader.dataset),
                100. * step / len(train_dataloader), loss))

def evaluate(loader):
  model.eval()

  n_correct, n_all = 0, 0

  for batch in loader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)
      logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    predictions = np.argmax(logits, axis=1)

    labels = b_labels.to('cpu').numpy()
    n_correct += np.sum(predictions == labels)
    n_all += len(labels)

  print('Accuracy: [{}/{}] {:.4f}\n'.format(n_correct,
                                            n_all,
                                            n_correct/n_all))

# Now we are ready to train our model using the train()
# function. After each epoch, we evaluate the model using the
# validation set and evaluate().

train_lossv = []
for epoch in range(1, EPOCHS + 1):
    train(epoch, train_lossv)
    print('\nValidation set:')
    evaluate(validation_dataloader)

# ## Inference
#
# For a better measure of the quality of the model, let's see the
# model accuracy for the test messages.

print('Test set:')
evaluate(test_dataloader)

Train Epoch: 1 [0/2702 (0%)]	Loss: 3.016967

Validation set:
Accuracy: [234/301] 0.7774

Train Epoch: 2 [0/2702 (0%)]	Loss: 0.647195

Validation set:
Accuracy: [174/301] 0.5781

Train Epoch: 3 [0/2702 (0%)]	Loss: 0.713558

Validation set:
Accuracy: [250/301] 0.8306

Train Epoch: 4 [0/2702 (0%)]	Loss: 0.505990

Validation set:
Accuracy: [253/301] 0.8405

Train Epoch: 5 [0/2702 (0%)]	Loss: 0.118088

Validation set:
Accuracy: [249/301] 0.8272

Train Epoch: 6 [0/2702 (0%)]	Loss: 0.038239

Validation set:
Accuracy: [254/301] 0.8439

Train Epoch: 7 [0/2702 (0%)]	Loss: 0.023799

Validation set:
Accuracy: [248/301] 0.8239

Train Epoch: 8 [0/2702 (0%)]	Loss: 0.054967

Validation set:
Accuracy: [242/301] 0.8040

Train Epoch: 9 [0/2702 (0%)]	Loss: 0.010712

Validation set:
Accuracy: [248/301] 0.8239

Train Epoch: 10 [0/2702 (0%)]	Loss: 0.219685

Validation set:
Accuracy: [246/301] 0.8173

Train Epoch: 11 [0/2702 (0%)]	Loss: 0.006934

Validation set:
Accuracy: [244/301] 0.8106

Train Epoch: 12 [0/